查船位_每30分鐘回報1次版


In [ ]:
import pandas as pd
import time
import collections
from datetime import datetime, date, time, timedelta
import os

empty_dict = collections.defaultdict(list)

def pursine(df, GPStime_column_name, vessel_column_name, vessel_list, check_time) :
    df2 = df.copy()
    df[GPStime_column_name] = pd.to_datetime(df[GPStime_column_name], format='%Y/%m/%d %H:%M:%S')
    df_group = df.groupby(vessel_column_name)
    for i in vessel_list:
        try:
            df = df_group.get_group(i).reset_index()
            print()
            print(i)

            for e in range(0,9999):  # 有成功
                try:
                    if df[GPStime_column_name][e]+timedelta(minutes=30) < df[GPStime_column_name][e+1] :
                        if df[GPStime_column_name][e]+timedelta(minutes=30)*2 < df[GPStime_column_name][e+1] :
                            # print(df[GPStime_column_name][e]+timedelta(minutes=30),df[GPStime_column_name][e]+timedelta(minutes=30)*2)
                            empty_dict[i].append(str(df[GPStime_column_name][e] + timedelta(minutes=30)))
                            empty_dict[i].append(str(df[GPStime_column_name][e] + timedelta(minutes=30)* 2))
                            for r in range(3,9999):
                                if df[GPStime_column_name][e]+timedelta(minutes=30)*r < df[GPStime_column_name][e+1] :
                                    #print(df[GPStime_column_name][e]+timedelta(minutes=30)*r)
                                    empty_dict[i].append(str(df[GPStime_column_name][e] + timedelta(minutes=30) * r))
                                else:
                                    continue
                    else:
                        continue
                except (ValueError,KeyError):
                    print('Done')
                    break
        except KeyError as e:
            print()
            print(i)
            print("NO Data")
            empty_dict[i].append('NO Data')


    print('----------------------------------------------------------------------------------------------------')

    df2[GPStime_column_name] = pd.to_datetime(df2[GPStime_column_name], format='%Y/%m/%d %H:%M:%S')

    df2_group = df2.groupby(vessel_column_name)

    for q in vessel_list:
        try:
            df3 = df2_group.get_group(q).reset_index()
            df3 = df3.sort_values(GPStime_column_name,ignore_index=True,ascending=False)
            print()
            print(q)
            for x in range(0,1):
                for r in range(2,9999):
                    if df3[GPStime_column_name][0]+timedelta(minutes=30) < datetime.strptime(check_time, "%Y-%m-%d") :
                        if df3[GPStime_column_name][0]+timedelta(minutes=30)*r < datetime.strptime(check_time, "%Y-%m-%d") :
                            # print(df3[GPStime_column_name][x]+timedelta(minutes=30)*r)
                            empty_dict[q].append(str(df3[GPStime_column_name][x]+timedelta(minutes=30)*r))
                            new_column_lst = []
                            new_column_lst.append({vessel_column_name:df2[vessel_column_name][0],GPStime_column_name:df2[GPStime_column_name][x]+timedelta(minutes=30)})
                            df3 = df3.append(new_column_lst)
                            df3 = df3.sort_values(GPStime_column_name,ignore_index=True,ascending=False)
            print('Done')
        except KeyError as e:
            print()
            print(q)
            print("NO Data")
            empty_dict[q].append('NO Data')
    return empty_dict

if __name__ == '__main__':
    vessel_list = list(("裕穩101", "裕穩301", "百富103", "百富301", "大慶666", "慶豐787", "慶豐767",
                        "日友168", "日友768", "日友668", "日友868", "日友968", "日友568", "豐國188", "豐國189", "豐國828", "豐國866",
                        "豐國889", "豐國688", "協豐788", "協豐789", "華偉707", "興富華707", "興成發707", "興華發707", "興旺發707", "興永發707",
                        "富冠808", "穩發626", "穩發666", "穩發618", "穩發636"))

    GPStime_column_name = 'GPS時間'
    vessel_column_name = '中文船名'
    check_time = '2021-10-28' # 輸入查詢日期
    df = pd.read_csv('./vessel.csv', encoding='big5', engine='python')
    pursine(df=df,GPStime_column_name=GPStime_column_name, vessel_column_name=vessel_column_name, vessel_list=vessel_list, check_time=check_time)
    # print(empty_dict)
    folderpath = "./vessel_record"  # 建立目錄用的
    datetime_dt = datetime.today()  # 獲得當地時間
    datetime_str = datetime_dt.strftime("%Y-%m-%d-%H-%M-%S")  # 格式化日期
    restore_time = datetime_str # 建立檔案名稱用的

    # 開始建立目錄
    try:
        os.makedirs(folderpath)
    # 檔案已存在的例外處理
    except FileExistsError:
        print("目錄已存在。")

    # 開始建立儲存資料的檔案名稱
    fname = './vessel_record/check_date_{time}.txt'.format(time=restore_time)

    with open(fname, 'a') as fp:
        number = 1
        for name in empty_dict:
            fp.write("\n\n")
            fp.write(name)
            fp.write("\n")
            for detail in empty_dict[name]:
                if number == 0:
                    fp.write(detail)
                    fp.write(',')
                    fp.write(" ")
                    number +=1
                elif detail == 'NO Data':
                    fp.write(detail)
                    fp.write(',')
                    fp.write(" ")
                    number = 1
                elif empty_dict[name].index(detail) == len(empty_dict[name]) - 1:
                    fp.write(detail)
                    number = 1
                elif number % 8 == 0:
                    fp.write(detail)
                    fp.write(',')
                    fp.write("\n")
                    number +=1
                else:
                    fp.write(detail)
                    fp.write(',')
                    fp.write(" ")
                    number +=1


裕穩101
NO Data

裕穩301
NO Data

百富103
Done

百富301
NO Data

大慶666
Done

慶豐787
Done

慶豐767
NO Data

日友168
Done

日友768
Done

日友668
NO Data

日友868
Done

日友968
Done

日友568
Done

豐國188
Done

豐國189
Done

豐國828
Done

豐國866
Done

豐國889
Done

豐國688
Done

協豐788
Done

協豐789
Done

華偉707
Done

興富華707
Done

興成發707
Done

興華發707
Done

興旺發707
Done

興永發707
Done

富冠808
Done

穩發626
Done

穩發666
Done

穩發618
Done

穩發636
Done
----------------------------------------------------------------------------------------------------

裕穩101
NO Data

裕穩301
NO Data

百富103
Done

百富301
NO Data

大慶666
Done

慶豐787
Done

慶豐767
NO Data

日友168
Done

日友768
Done

日友668
NO Data

日友868
Done

日友968
Done

日友568
Done

豐國188
Done

豐國189
Done

豐國828
Done

豐國866
Done

豐國889
Done

豐國688
Done

協豐788
Done

協豐789
Done

華偉707
Done

興富華707
Done

興成發707
Done

興華發707
Done

興旺發707
Done

興永發707
Done

富冠808
Done

穩發626
Done

穩發666
Done

穩發618
Done

穩發636
Done


查船位_每小時回報1次版

In [ ]:
import pandas as pd
import time
import collections
from datetime import datetime, date, time, timedelta
import os

empty_dict = collections.defaultdict(list)

def pursine(df, GPStime_column_name, vessel_column_name, vessel_list, check_time) :
    df2 = df.copy()
    df[GPStime_column_name] = pd.to_datetime(df[GPStime_column_name], format='%Y/%m/%d %H:%M:%S')
    df_group = df.groupby(vessel_column_name)
    for i in vessel_list:
        try:
            df = df_group.get_group(i).reset_index()
            print()
            print(i)

            for e in range(0,9999):  # 有成功
                try:
                    if df[GPStime_column_name][e]+timedelta(minutes=60) < df[GPStime_column_name][e+1] :
                        if df[GPStime_column_name][e]+timedelta(minutes=60)*2 < df[GPStime_column_name][e+1] :
                            # print(df[GPStime_column_name][e]+timedelta(minutes=60),df[GPStime_column_name][e]+timedelta(minutes=60)*2)
                            empty_dict[i].append(str(df[GPStime_column_name][e] + timedelta(minutes=60)))
                            empty_dict[i].append(str(df[GPStime_column_name][e] + timedelta(minutes=60)* 2))
                            for r in range(3,9999):
                                if df[GPStime_column_name][e]+timedelta(minutes=60)*r < df[GPStime_column_name][e+1] :
                                    #print(df[GPStime_column_name][e]+timedelta(minutes=60)*r)
                                    empty_dict[i].append(str(df[GPStime_column_name][e] + timedelta(minutes=60) * r))
                                else:
                                    continue
                    else:
                        continue
                except (ValueError,KeyError):
                    print('Done')
                    break
        except KeyError as e:
            print()
            print(i)
            print("NO Data")
            empty_dict[i].append('NO Data')


    print('----------------------------------------------------------------------------------------------------')

    df2[GPStime_column_name] = pd.to_datetime(df2[GPStime_column_name], format='%Y/%m/%d %H:%M:%S')

    df2_group = df2.groupby(vessel_column_name)

    for q in vessel_list:
        try:
            df3 = df2_group.get_group(q).reset_index()
            df3 = df3.sort_values(GPStime_column_name,ignore_index=True,ascending=False)
            print()
            print(q)
            for x in range(0,1):
                for r in range(2,9999):
                    if df3[GPStime_column_name][0]+timedelta(minutes=60) < datetime.strptime(check_time, "%Y-%m-%d") :
                        if df3[GPStime_column_name][0]+timedelta(minutes=60)*r < datetime.strptime(check_time, "%Y-%m-%d") :
                            # print(df3[GPStime_column_name][x]+timedelta(minutes=60)*r)
                            empty_dict[q].append(str(df3[GPStime_column_name][x]+timedelta(minutes=60)*r))
                            new_column_lst = []
                            new_column_lst.append({vessel_column_name:df2[vessel_column_name][0],GPStime_column_name:df2[GPStime_column_name][x]+timedelta(minutes=30)})
                            df3 = df3.append(new_column_lst)
                            df3 = df3.sort_values(GPStime_column_name,ignore_index=True,ascending=False)
            print('Done')
        except KeyError as e:
            print()
            print(q)
            print("NO Data")
            empty_dict[q].append('NO Data')
    return empty_dict

if __name__ == '__main__':
    vessel_list = list(("裕穩101", "裕穩301", "百富103", "百富301", "大慶666", "慶豐787", "慶豐767",
                        "日友168", "日友768", "日友668", "日友868", "日友968", "日友568", "豐國188", "豐國189", "豐國828", "豐國866",
                        "豐國889", "豐國688", "協豐788", "協豐789", "華偉707", "興富華707", "興成發707", "興華發707", "興旺發707", "興永發707",
                        "富冠808", "穩發626", "穩發666", "穩發618", "穩發636"))

    GPStime_column_name = 'GPS時間'
    vessel_column_name = '中文船名'
    check_time = '2021-09-26' # 輸入查詢日期
    df = pd.read_csv('./vessel.csv', encoding='big5', engine='python')
    pursine(df=df,GPStime_column_name=GPStime_column_name, vessel_column_name=vessel_column_name, vessel_list=vessel_list, check_time=check_time)
    # print(empty_dict)
    folderpath = "./vessel_record"  # 建立目錄用的
    datetime_dt = datetime.today()  # 獲得當地時間
    datetime_str = datetime_dt.strftime("%Y-%m-%d-%H-%M-%S")  # 格式化日期
    restore_time = datetime_str # 建立檔案名稱用的

    # 開始建立目錄
    try:
        os.makedirs(folderpath)
    # 檔案已存在的例外處理
    except FileExistsError:
        print("目錄已存在。")

    # 開始建立儲存資料的檔案名稱
    fname = './vessel_record/check_date_{time}.txt'.format(time=restore_time)

    with open(fname, 'a') as fp:
        number = 1
        for name in empty_dict:
            fp.write("\n\n")
            fp.write(name)
            fp.write("\n")
            for detail in empty_dict[name]:
                if number == 0:
                    fp.write(detail)
                    fp.write(',')
                    fp.write(" ")
                    number +=1
                elif detail == 'NO Data':
                    fp.write(detail)
                    fp.write(',')
                    fp.write(" ")
                    number = 1
                elif empty_dict[name].index(detail) == len(empty_dict[name]) - 1:
                    fp.write(detail)
                    number = 1
                elif number % 8 == 0:
                    fp.write(detail)
                    fp.write(',')
                    fp.write("\n")
                    number +=1
                else:
                    fp.write(detail)
                    fp.write(',')
                    fp.write(" ")
                    number +=1

查公海

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import warnings
import os

def check_non_EEZ(df):
  df['GPS時間'] = pd.to_datetime(df['GPS時間'], format='%Y/%m/%d %H:%M:%S')
  df_na = df[df['EEZ'].isnull()]
  df_na['GPS時間'] = df_na['GPS時間'].map(lambda x : x.strftime("%Y/%m/%d"))
  df_na['EEZ'] = df_na['EEZ'].fillna('non EEZ')
  df_detail = df_na[['中文船名','GPS時間','EEZ']].groupby(['中文船名','GPS時間']).count().reset_index()
  df_detail = df_detail.rename(columns={'EEZ':'非EEZ次數'})
  df_detail = df_detail.sort_values('GPS時間')
  df_detail.to_csv(fname,index=False, encoding='big5')
  return df_detail

  
if __name__ == '__main__':
  warnings.simplefilter('ignore')
  df = pd.read_csv('./vessel.csv', encoding='big5', engine='python')
  check_non_EEZ(df=df)
  folderpath = "./vessel_record"  # 建立目錄用的
  datetime_dt = datetime.today()  # 獲得當地時間
  datetime_str = datetime_dt.strftime("%Y-%m-%d-%H-%M-%S")  # 格式化日期
  restore_time = datetime_str # 建立檔案名稱用的
  # 開始建立目錄
  try:
    os.makedirs(folderpath)
  # 檔案已存在的例外處理
  except FileExistsError:
    print("目錄已存在。")
  
  fname = './vessel_record/check_High_Sea_{time}.csv'.format(time=restore_time)
  print('Done') 
  

目錄已存在。
Done
